# Why should you work with this type of dataset
If you work on this type of dataset, you can implmenet similar solutions in any IoT related project in your organization or personal project. 

This dataset can help you to learn:
    - how to approach sensors data
    - how to find anomaly which can help you to know when device is going to break
    - It do have timeseries angle, so could look for that too. 

# Step 1. 
 - Have a quick look on the dataset, which is very much needed to build the thought process around the data
 - Reread problem statement multiple times and try to understand how to correlate the dataset and problem. 
 - Plot the visualization 
 
 Our problem statment: Look for the correlation of sensors which leads to device breakdown



1. Quick Checklist for this dataset
*     timeseries forecasting problem 
*     machine status - 3 convert that into label encoding 
*     all numerical values
*     anomaly detection 
*     labelled data - supervised learning , classification 
*     look for correlation matrix 
*     look for skewness in the data 
*     check imbalance data if any 

In [30]:
# load the dataset 
import pandas as pd
import numpy as np

df = pd.read_csv('../input/pumpsensor/sensor.csv')
df.head(2)

In [31]:
df.tail()

# 01-Apr-2018 to 31-Aug-2018 
# apr, may, jun, jul, aug - 5 months every min data is collected 

# "Unnamed" column removed (deleted).

In [32]:
del df['Unnamed: 0']

In [33]:
# change timestamp column into the index column
df['index'] = pd.to_datetime(df['timestamp'])
df.index = df['index']

# Again, looking at the row titles

In [34]:
df.head(2)

In [35]:
# delete the colunmns. Now, the timestamp is in the index column
del df['index']
del df['timestamp']

In [36]:
df.head(2)

In [37]:
df['sensor_15'].nunique() # no unique - complete zero
# drop the column 
df.drop(['sensor_15'], axis=1, inplace = True)
df.shape

In [38]:
df.info()

In [39]:
# machine status - no null 
# we will drop na in whole dataframe 
df['sensor_00'].isna().sum()

# Request number of counts for 'NORMAL', 'BROKEN', 'RECOVERING' 

In [40]:
# machine status
df['machine_status'].unique()#'NORMAL', 'BROKEN', 'RECOVERING' 
df['machine_status'].value_counts()

In [41]:
# draw a countplot for machine status 
import seaborn as sns
sns.countplot(y = df['machine_status'])

** Steps to discover the metrics that identify device failures, 
* it is highly imbalanced data  and undersampling can't help here 
* next, is an experiment with SMOTE or oversampling 

In [42]:
# apply label encoder to encode the machine status
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df['machine_status'] = le.fit_transform(df['machine_status'])
df['machine_status'].value_counts()

# 1 - normal 
# 2 - recovering 
# 0 - broken

In [43]:
#  look on complete data frame when device is broken
df_broken = df[df.machine_status ==0]
df_broken

# there is no nan value corellation for broken device 
#

In [44]:
import matplotlib.pyplot as plt 
plt.plot(df['sensor_02'])

In [45]:
# imputation for null values 
df['sensor_04'].hist()
# data is skewed so we need to use median value to fill the data

In [46]:
# let us figureout NaN values 
df['sensor_00'].isna().sum()

In [47]:
df['sensor_50'].isna().sum()

In [48]:
# used ffill method to fill the missing values
df = df.fillna(method='ffill')

In [49]:
X = df.drop(['machine_status'], axis=1)
X.shape

In [50]:
Y = df['machine_status']
Y.shape

In [51]:
# apply the logistic regression 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size = 0.30, random_state = 42)

In [52]:
# apply 
logit = LogisticRegression()
model = logit.fit(X_train, y_train)

In [53]:
# predict
y_pred = model.predict(X_test)

In [54]:
# evaluate the model
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
cm = pd.crosstab(y_test,y_pred, rownames=['True'], colnames=['Predicted'], margins=True)
cm

In [55]:
# accuracy is not a good metrics for Anomaly detection and imbalanced dataset
accuracy = accuracy_score(y_test, y_pred)
accuracy

In [56]:
# Classification Report
cr = classification_report(y_pred, y_test)
print(cr)

# these reports are not good 
we will use Isolation forest and oneSVM for modelling 
xgboosting feature_importance and PCA for dimension reduction 
before that we will divide this dataset into 2 problems
 machine status - normal + broken, normal + recovery, recovery+ broken 

In [57]:
df.shape # look on the shape of the dataset

 machine status - normal + broken

In [58]:
df1 = df.copy()
df1 = df[(df1.machine_status ==1) | (df1.machine_status ==0)]
df1.shape